# Objetivo

Criar um sistema de ranqueamento para conseguir classificar as concessionárias baseado na [Resolução n° 6.053 da ANTT](https://www.in.gov.br/en/web/dou/-/resolucao-n-6.053-de-31-de-outubro-de-2024-593328784)

## Fórmula da Nota global
$$
NG = 0,20 \left( \frac{\overline{P_{IRI}} + \overline{P_{DC}} + \overline{P_{TR}} + \overline{P_{IFI}} + \overline{P_{\%TRIN}}}{5} \right) + 0,10 \left[ 0,5 \overline{S_{RSH}} + 0,5 \left( \frac{\overline{S_{RSV}} + \overline{S_{SSV}} + \overline{S_{CSV}}}{3} \right) \right] + 0,05 \left( \frac{\overline{OAC_{DS}} + \overline{OAC_{DP}}}{2} \right) + 0,05 \overline{OAE} + 0,025 \overline{IDA} + 0,05 \overline{O_{EAn}} + 0,25 \overline{O_{EAc}} + 0,10 \left( \frac{\overline{U_{IS}} + \overline{U_{SA}} + \overline{U_{RR}} + \overline{U_{PR}}}{4} \right) + 0,10 \overline{U_{SMéd}} + 0,10 \overline{U_{SMec}}
$$


In [11]:
!pip install numpy pandas

In [12]:
from dataclasses import dataclass
from typing import Literal, Dict, Tuple
import pandas as pd
import numpy as np

In [13]:
@dataclass(frozen=True)
class Pesos:
  # weight = pesos. como geralmente é usado w para coeficiente de vetor, vou usar aqui também
  w_pav: float = 0.20    # bloco Pavimento (média dos 5 subindicadores)
  w_sin: float = 0.10    # bloco Sinalização (0,5*RSH + 0,5*média(RSV,SSV,CSV))
  w_dren: float = 0.05   # média(OAC_DS, OAC_DP)
  w_oae: float = 0.05    # OAE
  w_ida: float = 0.025   # IDA
  w_ean: float = 0.05    # O_EAn
  w_eac: float = 0.25    # O_EAC
  w_ouv: float = 0.10    # média(U_IS, U_SA, U_RR, U_PR)
  w_smed: float = 0.10   # U_SMed
  w_smec: float = 0.10   # U_SMec

  def soma(self) -> float:
    return (self.w_pav + self.w_sin + self.w_dren + self.w_oae + self.w_ida + self.w_ean + self.w_eac + self.w_ouv + self.w_smed + self.w_smec)

PESOS = Pesos()

## Calcular a nota geral

In [14]:
# função auxiliar só para limitar as notas ao intervalo de 0 a 10
def _clip01(x: pd.Series | float, clip_0_10: bool) -> pd.Series | float:
    return np.clip(x, 0, 10) if clip_0_10 else x

def calcular_ng(df: pd.DataFrame, pesos: Pesos = PESOS, clip_0_10: bool = True) -> pd.Series:
    """
    Calcula a Nota Global (NG) linha a linha.
    Retorna uma pd.Series com a NG.
    """
    # Pavimento: média simples de 5 subindicadores
    pav = (_clip01(df['P_IRI'], clip_0_10) + _clip01(df['P_DC'], clip_0_10) +
           _clip01(df['P_TR'], clip_0_10) + _clip01(df['P_IFI'], clip_0_10) +
           _clip01(df['P_pctTRIN'], clip_0_10)) / 5.0

    # Sinalização: 0,5*RSH + 0,5*média(RSV,SSV,CSV)
    sin_media_placas = (_clip01(df['S_RSV'], clip_0_10) +
                        _clip01(df['S_SSV'], clip_0_10) +
                        _clip01(df['S_CSV'], clip_0_10)) / 3.0
    sin = 0.5 * _clip01(df['S_RSH'], clip_0_10) + 0.5 * sin_media_placas

    # Drenagem (correntes): média simples
    dren = (_clip01(df['OAC_DS'], clip_0_10) + _clip00(df['OAC_DP'], clip_0_10)) / 2.0

    # Ouvidoria: média dos quatro subindicadores
    ouv = (_clip01(df['U_IS'], clip_0_10) + _clip01(df['U_SA'], clip_0_10) +
           _clip01(df['U_RR'], clip_0_10) + _clip01(df['U_PR'], clip_0_10)) / 4.0

    # Termos isolados
    oae  = _clip01(df['OAE'], clip_0_10)
    ida  = _clip01(df['IDA'], clip_0_10)
    ean  = _clip01(df['O_EAn'], clip_0_10)
    eac  = _clip01(df['O_EAC'], clip_0_10)
    smed = _clip01(df['U_SMed'], clip_0_10)
    smec = _clip01(df['U_SMec'], clip_0_10)

    ng = (pesos.w_pav  * pav +
          pesos.w_sin  * sin +
          pesos.w_dren * dren +
          pesos.w_oae  * oae +
          pesos.w_ida  * ida +
          pesos.w_ean  * ean +
          pesos.w_eac  * eac +
          pesos.w_ouv  * ouv +
          pesos.w_smed * smed +
          pesos.w_smec * smec)

    # #Imprima a soma dos pesos para auditoria
    if not np.isclose(pesos.soma(), 1.0, atol=1e-6):
        print(f"[AVISO] Soma dos pesos = {pesos.soma():.3f} (≠ 1.000).")

    return ng

## Classificação A/B/C/D

Além da numérica, eles também usam uma equivalente mas usando letras

In [15]:
TipoAno = Literal["1", "2", "3", "4+"]

def classificar_ng(ng: float, ano_classificacao: TipoAno) -> str:
    """
    Retorna A/B/C/D LEVANDO EM CONSIDERAÇÃO o tempo que a fiscalização está sendo realizada.
    """
    if ano_classificacao == "1":
        if ng >= 8.0:             return "A"
        if 5.5 <= ng < 8.0:       return "B"
        if 3.5 <= ng < 5.5:       return "C"
        return "D"                # ng < 3.5

    if ano_classificacao == "2":
        if ng > 8.0:              return "A"
        if 6.0 <= ng <= 8.0:      return "B"
        if 4.0 < ng < 6.0:        return "C"
        return "D"                # ng <= 4.0

    if ano_classificacao == "3":
        if ng >= 8.5:             return "A"
        if 6.5 <= ng < 8.5:       return "B"
        if 4.5 < ng < 6.5:        return "C"
        return "D"                # ng <= 4.5

    # "4+" (a partir do quarto ano)
    if ng >= 8.5:                 return "A"
    if 7.0 <= ng < 8.5:           return "B"
    if 5.0 <= ng < 7.0:           return "C"
    return "D"                    # ng < 5.0

In [16]:
# só jogar o dicionário em pandas e o ano de classificação
def ranquear(df: pd.DataFrame, ano_classificacao: TipoAno, pesos: Pesos = PESOS, clip_0_10: bool = True) -> pd.DataFrame:
    """
    Adiciona colunas: NG, Classe, Rank (ordem decrescente da NG).
    """
    df = df.copy()
    df['NG'] = calcular_ng(df, pesos=pesos, clip_0_10=clip_0_10)
    df['Classe'] = df['NG'].apply(lambda x: classificar_ng(float(x), ano_classificacao))
    df['Rank'] = df['NG'].rank(method='dense', ascending=False).astype(int)
    return df.sort_values(['NG','concessionaria'], ascending=[False, True]).reset_index(drop=True)